In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# CUDA_VISIBLE_DEVICES=0 python run.py train --train-src=./en_es_data/train.es --train-tgt=./en_es_data/train.en --dev-src=./en_es_data/dev.es --dev-tgt=./en_es_data/dev.en --vocab=vocab.json --cuda

In [77]:
import math
import sys
import time

import numpy as np
import torch

from utils import read_corpus, batch_iter
from nmt_model import Hypothesis, NMT
from vocab import Vocab

### params

In [79]:
# architecture
embed_size = 256
hidden_size = 256
dropout = 0.3
uniform_init = 0.1

# training
cuda = False
lr_arg = 0.001
lr_decay = 0.5
patience_arg = 5  # wait for how many iterations to decay learning rate
max_num_trial = 5  # terminate training after how many trials
train_batch_size = 32
clip_grad = 5.0
valid_niter = 2000
log_every = 10
max_epoch = 30
model_save_path = 'model.bin'
train_src = '/Users/aziai/Dropbox/github/cs224n-nlp/assignments/assignment4/en_es_data/train.es'
train_tgt = '/Users/aziai/Dropbox/github/cs224n-nlp/assignments/assignment4/en_es_data/train.en'
dev_src = '/Users/aziai/Dropbox/github/cs224n-nlp/assignments/assignment4/en_es_data/dev.es'
dev_tgt = '/Users/aziai/Dropbox/github/cs224n-nlp/assignments/assignment4/en_es_data/dev.en'
file_path_vocab = '/Users/aziai/Dropbox/github/cs224n-nlp/assignments/assignment4/vocab.json'

### run

In [ ]:
v = vocab.build()

In [83]:
train_data_src[4]

['Ahora',
 'tengo',
 'que',
 'quitarme',
 'mis',
 'zapatos',
 'o',
 'botas',
 'para',
 'subirme',
 'a',
 'un',
 'avin!']

In [67]:
train_data_src = read_corpus(train_src, source='src')
train_data_tgt = read_corpus(train_tgt, source='tgt')

dev_data_src = read_corpus(dev_src, source='src')
dev_data_tgt = read_corpus(dev_tgt, source='tgt')

train_data = list(zip(train_data_src, train_data_tgt))
dev_data = list(zip(dev_data_src, dev_data_tgt))

In [68]:
vocab = Vocab.load(file_path_vocab)
model = NMT(embed_size=embed_size, hidden_size=hidden_size, dropout_rate=dropout,
            vocab=vocab)

In [108]:
train_data_src

[['Muchas',
  'gracias',
  'Chris.',
  'Y',
  'es',
  'en',
  'verdad',
  'un',
  'gran',
  'honor',
  'tener',
  'la',
  'oportunidad',
  'de',
  'venir',
  'a',
  'este',
  'escenario',
  'por',
  'segunda',
  'vez.',
  'Estoy',
  'extremadamente',
  'agradecido.'],
 ['He',
  'quedado',
  'conmovido',
  'por',
  'esta',
  'conferencia,',
  'y',
  'deseo',
  'agradecer',
  'a',
  'todos',
  'ustedes',
  'sus',
  'amables',
  'comentarios',
  'acerca',
  'de',
  'lo',
  'que',
  'tena',
  'que',
  'decir',
  'la',
  'otra',
  'noche.'],
 ['Y',
  'digo',
  'eso',
  'sinceramente,',
  'en',
  'parte',
  'porque',
  '--',
  '(Sollozos',
  'fingidos)',
  '--',
  'lo',
  'necesito!',
  '',
  'Pnganse',
  'en',
  'mi',
  'posicin!'],
 ['Vol', 'en', 'el', 'avin', 'vicepresidencial', 'por', 'ocho', 'aos.'],
 ['Ahora',
  'tengo',
  'que',
  'quitarme',
  'mis',
  'zapatos',
  'o',
  'botas',
  'para',
  'subirme',
  'a',
  'un',
  'avin!'],
 ['Les',
  'dir',
  'una',
  'rpida',
  'historia',
  

In [85]:
vocab.build(train_data_src, train_data_tgt, 1000, 1)

initialize source vocabulary ..
number of word types: 172418, number of word types w/ frequency >= 1: 172418
initialize target vocabulary ..
number of word types: 128873, number of word types w/ frequency >= 1: 128873


Vocab(source 1004 words, target 1002 words)

In [105]:
model.eval

<bound method Module.eval of NMT(
  (model_embeddings): ModelEmbeddings(
    (source): Embedding(50004, 256, padding_idx=0)
    (target): Embedding(50002, 256, padding_idx=0)
  )
  (encoder): LSTM(256, 256, bidirectional=True)
  (decoder): LSTMCell(512, 256)
  (h_projection): Linear(in_features=512, out_features=256, bias=False)
  (c_projection): Linear(in_features=512, out_features=256, bias=False)
  (att_projection): Linear(in_features=512, out_features=256, bias=False)
  (combined_output_projection): Linear(in_features=768, out_features=256, bias=False)
  (target_vocab_projection): Linear(in_features=256, out_features=50002, bias=False)
  (dropout): Dropout(p=0.3)
)>

In [93]:
time.time()

1552420795.128779

In [88]:
type(train_data[0])

tuple

In [91]:
list(zip(*train_data))[1]

(['<s>',
  'Thank',
  'you',
  'so',
  'much,',
  'Chris.',
  'And',
  "it's",
  'truly',
  'a',
  'great',
  'honor',
  'to',
  'have',
  'the',
  'opportunity',
  'to',
  'come',
  'to',
  'this',
  'stage',
  'twice;',
  "I'm",
  'extremely',
  'grateful.',
  '</s>'],
 ['<s>',
  'I',
  'have',
  'been',
  'blown',
  'away',
  'by',
  'this',
  'conference,',
  'and',
  'I',
  'want',
  'to',
  'thank',
  'all',
  'of',
  'you',
  'for',
  'the',
  'many',
  'nice',
  'comments',
  'about',
  'what',
  'I',
  'had',
  'to',
  'say',
  'the',
  'other',
  'night.',
  '</s>'],
 ['<s>',
  'And',
  'I',
  'say',
  'that',
  'sincerely,',
  'partly',
  'because',
  '(Mock',
  'sob)',
  'I',
  'need',
  'that.',
  '',
  'Put',
  'yourselves',
  'in',
  'my',
  'position.',
  '</s>'],
 ['<s>',
  'I',
  'flew',
  'on',
  'Air',
  'Force',
  'Two',
  'for',
  'eight',
  'years.',
  '</s>'],
 ['<s>',
  'Now',
  'I',
  'have',
  'to',
  'take',
  'off',
  'my',
  'shoes',
  'or',
  'boots',
  '

In [69]:
model.train()

NMT(
  (model_embeddings): ModelEmbeddings(
    (source): Embedding(50004, 256, padding_idx=0)
    (target): Embedding(50002, 256, padding_idx=0)
  )
  (encoder): LSTM(256, 256, bidirectional=True)
  (decoder): LSTMCell(512, 256)
  (h_projection): Linear(in_features=512, out_features=256, bias=False)
  (c_projection): Linear(in_features=512, out_features=256, bias=False)
  (att_projection): Linear(in_features=512, out_features=256, bias=False)
  (combined_output_projection): Linear(in_features=768, out_features=256, bias=False)
  (target_vocab_projection): Linear(in_features=256, out_features=50002, bias=False)
  (dropout): Dropout(p=0.3)
)

In [70]:
len(list(model.parameters()))

19

In [71]:
# initalize params
if np.abs(uniform_init) > 0.:
    print('uniformly initialize parameters [-%f, +%f]' % (uniform_init, uniform_init), file=sys.stderr)
    for p in model.parameters():
        p.data.uniform_(-uniform_init, uniform_init)

uniformly initialize parameters [-0.100000, +0.100000]


In [72]:
# TODO: ?
vocab_mask = torch.ones(len(vocab.tgt))
vocab_mask[vocab.tgt['<pad>']] = 0
vocab_mask.shape

torch.Size([50002])

In [73]:
device = torch.device("cuda:0" if cuda else "cpu")
print('use device: %s' % device, file=sys.stderr)
model = model.to(device)

use device: cpu


In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_arg)

In [75]:
num_trial = 0
train_iter = patience = cum_loss = report_loss = cum_tgt_words = report_tgt_words = 0
cum_examples = report_examples = epoch = valid_num = 0
hist_valid_scores = []
train_time = begin_time = time.time()
print('begin Maximum Likelihood training')

begin Maximum Likelihood training


In [78]:
while True:
    epoch += 1

    # TODO: apply the agumenter
    for src_sents, tgt_sents in batch_iter(train_data, batch_size=train_batch_size, shuffle=True):
        train_iter += 1

        optimizer.zero_grad()

        batch_size = len(src_sents)

        # apply the model to src and tgt
        example_losses = -model(src_sents, tgt_sents)  # (batch_size,)
        batch_loss = example_losses.sum()
        loss = batch_loss / batch_size

        loss.backward()

        # clip gradient
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        optimizer.step()

        batch_losses_val = batch_loss.item()
        report_loss += batch_losses_val
        cum_loss += batch_losses_val

        tgt_words_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
        report_tgt_words += tgt_words_num_to_predict
        cum_tgt_words += tgt_words_num_to_predict
        report_examples += batch_size
        cum_examples += batch_size

        if train_iter % log_every == 0:
            print('epoch %d, iter %d, avg. loss %.2f, avg. ppl %.2f ' \
                  'cum. examples %d, speed %.2f words/sec, time elapsed %.2f sec' % (epoch, train_iter,
                                                                                     report_loss / report_examples,
                                                                                     math.exp(
                                                                                         report_loss / report_tgt_words),
                                                                                     cum_examples,
                                                                                     report_tgt_words / (
                                                                                                 time.time() - train_time),
                                                                                     time.time() - begin_time),
                  file=sys.stderr)

            train_time = time.time()
            report_loss = report_tgt_words = report_examples = 0.

        # perform validation
        if train_iter % valid_niter == 0:
            print('epoch %d, iter %d, cum. loss %.2f, cum. ppl %.2f cum. examples %d' % (epoch, train_iter,
                                                                                         cum_loss / cum_examples,
                                                                                         np.exp(
                                                                                             cum_loss / cum_tgt_words),
                                                                                         cum_examples),
                  file=sys.stderr)

            cum_loss = cum_examples = cum_tgt_words = 0.
            valid_num += 1

            print('begin validation ...', file=sys.stderr)

            # compute dev. ppl and bleu
            dev_ppl = evaluate_ppl(model, dev_data, batch_size=128)  # dev batch size can be a bit larger
            valid_metric = -dev_ppl

            print('validation: iter %d, dev. ppl %f' % (train_iter, dev_ppl), file=sys.stderr)

            is_better = len(hist_valid_scores) == 0 or valid_metric > max(hist_valid_scores)
            hist_valid_scores.append(valid_metric)

            if is_better:
                patience = 0
                print('save currently the best model to [%s]' % model_save_path, file=sys.stderr)
                model.save(model_save_path)

                # also save the optimizers' state
                torch.save(optimizer.state_dict(), model_save_path + '.optim')
            elif patience < patience_arg:
                patience += 1
                print('hit patience %d' % patience, file=sys.stderr)

                if patience == patience_arg:
                    num_trial += 1
                    print('hit #%d trial' % num_trial, file=sys.stderr)
                    if num_trial == max_num_trial:
                        print('early stop!', file=sys.stderr)
                        exit(0)

                    # decay lr, and restore from previously best checkpoint
                    lr = optimizer.param_groups[0]['lr'] * float(lr_decay)
                    print('load previously best model and decay learning rate to %f' % lr, file=sys.stderr)

                    # load model
                    params = torch.load(model_save_path, map_location=lambda storage, loc: storage)
                    model.load_state_dict(params['state_dict'])
                    model = model.to(device)

                    print('restore parameters of the optimizers', file=sys.stderr)
                    optimizer.load_state_dict(torch.load(model_save_path + '.optim'))

                    # set new lr
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr

                    # reset patience
                    patience = 0

            if epoch == max_epoch:
                print('reached maximum number of epochs!', file=sys.stderr)
                exit(0)

epoch 2, iter 20, avg. loss 159.69, avg. ppl 7801.77 cum. examples 640, speed 111.03 words/sec, time elapsed 102.72 sec
epoch 2, iter 30, avg. loss 128.32, avg. ppl 1905.38 cum. examples 960, speed 301.55 words/sec, time elapsed 120.75 sec


KeyboardInterrupt: 

In [11]:
!head $file_path

Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.
I have been blown away by this conference, and I want to thank all of you for the many nice comments about what I had to say the other night.
And I say that sincerely, partly because (Mock sob) I need that.  Put yourselves in my position.
I flew on Air Force Two for eight years.
Now I have to take off my shoes or boots to get on an airplane!
I'll tell you one quick story to illustrate what that's been like for me.
It's a true story -- every bit of this is true.
Soon after Tipper and I left the -- (Mock sob) White House --  we were driving from our home in Nashville to a little farm we have 50 miles east of Nashville.
Driving ourselves.
I know it sounds like a little thing to you, but --  I looked in the rear-view mirror and all of a sudden it just hit me. There was no motorcade back there.


In [14]:
read_corpus(file_path, 'tgt')[0]

['<s>',
 'Thank',
 'you',
 'so',
 'much,',
 'Chris.',
 'And',
 "it's",
 'truly',
 'a',
 'great',
 'honor',
 'to',
 'have',
 'the',
 'opportunity',
 'to',
 'come',
 'to',
 'this',
 'stage',
 'twice;',
 "I'm",
 'extremely',
 'grateful.',
 '</s>']

In [16]:
read_corpus(file_path, 'src')[0]

['Thank',
 'you',
 'so',
 'much,',
 'Chris.',
 'And',
 "it's",
 'truly',
 'a',
 'great',
 'honor',
 'to',
 'have',
 'the',
 'opportunity',
 'to',
 'come',
 'to',
 'this',
 'stage',
 'twice;',
 "I'm",
 'extremely',
 'grateful.']